In [16]:
import pandas as pd
import json
import numpy as np
def load_jsonl(jsonl_fpath):
    with open(jsonl_fpath) as f:
        lines = f.read().splitlines()

    df = pd.DataFrame(lines)
    df.columns = ['json_element']
    df['json_element'].apply(json.loads)
    df = pd.json_normalize(df['json_element'].apply(json.loads))

    return df

df = load_jsonl('../data/mednli-1.0.0/mli_test_v1.jsonl')
df = df[['sentence1','sentence2','gold_label']]
df.head()

,sentence1,sentence2,gold_label
0,"In the ED, initial VS revealed T 98.9, HR 73, ...",The patient is hemodynamically stable,entailment
1,"In the ED, initial VS revealed T 98.9, HR 73, ...",The patient is hemodynamically unstable.,contradiction
2,"In the ED, initial VS revealed T 98.9, HR 73, ...",The patient is in pain.,neutral
3,63 year old woman with known history of hyperc...,the patient was in a MVC,entailment
4,63 year old woman with known history of hyperc...,the patient has no medical history,contradiction


In [25]:
classes = list(np.unique(df['gold_label']))
classes

['contradiction', 'entailment', 'neutral']

In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", return_dict=True)

'''
The goal of the task is to classify a given premise-hypothesis pair into one of the three classes: entailment, contradiction, or neutral.
'''

classes = list(np.unique(df['gold_label']))

for i in range(len(df)):
    premise = df.iloc[i]['sentence1']
    hypothesis = df.iloc[i]['sentence2']

    true_label = df.iloc[i]['gold_label']
        
    tokenized_sequences = tokenizer(premise, hypothesis, return_tensors="pt")

    logits = model(**tokenized_sequences).logits
    result = torch.softmax(logits, dim=1).tolist()[0]

    for c in range(len(classes)):
        print(f"{classes[c]}: {int(round(result[c] * 100))}%")
        # if true_label == 'entailment':
        # elif label == 'contradiction':
        # elif label == 'neutral':

    # Should not be contradiction
    # for c in range(len(classes)):
    #     print(f"{classes[c]}: {int(round(not_paraphrase_results[c] * 100))}%")
    # # Should be neutral
    # for c in range(len(classes)):
    #     print(f"{classes[c]}: {int(round(not_paraphrase_results[c] * 100))}%")

    
# paraphrase_classification_logits = model(**paraphrase).logits
# not_paraphrase_classification_logits = model(**not_paraphrase).logits

# paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
# not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

Should be entailment
for c in range(len(classes)):
    print(f"{classes[c]}: {int(round(paraphrase_results[c] * 100))}%")
# Should not be contradiction
for c in range(len(classes)):
    print(f"{classes[c]}: {int(round(not_paraphrase_results[c] * 100))}%")
# Should be neutral
for c in range(len(classes)):
    print(f"{classes[c]}: {int(round(not_paraphrase_results[c] * 100))}%")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

IndexError: list index out of range